# Elasticity equation - compute CFF

In this tutorial we present how to solve the elasticity equation with [PorePy](https://github.com/pmgbergen/porepy) and then how to post process the tractions to compute the CFF (Coulomb friction failure) coefficient along a fracture. The unknown is the displacement $u$.

Let $\Omega=(0,1)^3$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, we want to solve the following problem: find $u$ such that
$$
\nabla \cdot [ 2 \mu \epsilon(u) + \lambda \nabla \cdot u] = -b
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. The CFF is computed as
$$
    CFF = \tau^\top \sigma n + \mu_f n^\top \sigma n
$$
with $\mu_f$ the fracture friction coefficient. We have the following possibilities
- No slip: $CFF < 0$
- Slipping: $CFF \geq 0$

We will use the Multi-Point Stress Approximation (MPSA) to discretise the problem.

## Exercise 2: dam filling

The dam filling problem is when a (time dependent) force is impose on the top compressing the body, the bottom is fixed, and lateral compressive forces are imposed on the other sides of the boundary.

For this test case we set $\Omega = [0, 1]^3$, $b = 0$, and the following boundary conditions:
$$ 
u = 0 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot \sigma = [f_{lrfb}, 0] \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \cup \partial_{front}\Omega \cup \partial_{back} \Omega \qquad \nu \cdot \sigma = [0, f_t]^\top \text{ on } \partial_{top} \Omega
$$

whith $f_{lrfb} < 0 $ as well as $f_t < 0$.

We have the following relations between the $\lambda$ and $\mu$ as functions of the Young's modulus $E$ and Poisson ratio $\nu$
$$
\lambda = \dfrac{\nu E}{(1+\nu)(1-2\nu)}\qquad \mu = \dfrac{E}{2(1+\nu)}
$$

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid and the fracture/fault now represent an internal constraint so that the grid adapts to it.

In [2]:
N = 10
dim = 3
mesh_size = 1 / N

# Define a fracture
frac_pts = np.array(
    [
        [0.2, 0.9, 0.9, 0.2],
        [0.2, 0.2, 0.8, 0.8],
        [0.2, 1, 1, 0.2],
    ]
)
frac = pp.PlaneFracture(frac_pts)

sd = pg.unit_grid(dim, mesh_size, as_mdg=False, fractures=[frac], constraints=[0])
sd.compute_geometry()

# sd_exp = pg.unit_grid(dim, mesh_size, as_mdg=True, fractures=[frac])
# sd_exp.compute_geometry()
# pp.Exporter(sd_exp, "grid", folder_name="ex2").write_vtu()

With the following code we set the data, in particular the Lamé and the Kirchhoff modulus, and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "elasticity"

E = 1
nu = 0.25
mu_fric = 0.45

# forces imposed at the boundary
fun_top = -1e-2  # -1e-2 (no slip) or -2.5e-2 (some slip) or -3e-2 (slip)
fun_left = 1e-2
fun_right = -1e-2
fun_front = -1e-2
fun_back = 1e-2

lambda_ = E * nu / ((1 + nu) * (1 - 2 * nu))
mu = E / (2 * (1 + nu))

# Create stiffness matrix
lambda_ = lambda_ * np.ones(sd.num_cells)
mu = mu * np.ones(sd.num_cells) / 2
C = pp.FourthOrderTensor(mu, lambda_)

# Define boundary type
b_faces = sd.get_all_boundary_faces()
num_b_faces = b_faces.size
labels = np.array(["neu"] * num_b_faces)

bottom = np.isclose(sd.face_centers[2, b_faces], 0)
labels[bottom] = "dir"
bound = pp.BoundaryConditionVectorial(sd, b_faces, labels)

bc_values = np.zeros((sd.dim, sd.num_faces))

# force from the top of the domain
top = np.isclose(sd.face_centers[2, :], 1)
bc_values[2, top] = fun_top * sd.face_areas[top]

# compressive force on the left, right, front, and back boundaries
left = np.isclose(sd.face_centers[0, :], 0)
bc_values[0, left] = fun_left * sd.face_areas[left]

right = np.isclose(sd.face_centers[0, :], 1)
bc_values[0, right] = fun_right * sd.face_areas[right]

front = np.isclose(sd.face_centers[1, :], 0)
bc_values[1, front] = fun_front * sd.face_areas[front]

back = np.isclose(sd.face_centers[1, :], 1)
bc_values[1, back] = fun_back * sd.face_areas[back]

bc_values = bc_values.ravel("F")

# No source term
source = np.zeros(sd.num_cells * sd.dim)

# collect all data
param = {
    "fourth_order_tensor": C,
    "bc_values": bc_values,
    "bc": bound,
    "source": source,
}
data = pp.initialize_data({}, key, param)

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [4]:
# discretize and solve the system
mpsa = pp.Mpsa(key)
mpsa.discretize(sd, data)

A, b = mpsa.assemble_matrix_rhs(sd, data)
u = sps.linalg.spsolve(A, b)

We first compute the traction for each face of the cell and reshape it so that it has the x-components for all the faces first and then all the y-components (and then all z-components in 3d case).

In [5]:
# post process the traction for each face
mat = data[pp.DISCRETIZATION_MATRICES][key]
mat_stress = mat[mpsa.stress_matrix_key]
mat_bound_stress = mat[mpsa.bound_stress_matrix_key]

# The measure is in Pascals
t = mat_stress @ u + mat_bound_stress @ bc_values

# reshape the traction to be in the order of first all the x-components, then all the
# y-components, and finally the z-components
t = np.reshape(t, (sd.dim, -1), order="F").ravel()

We restrict the traction to the fracture faces

In [6]:
# compute the faces that are on the fracture
points = sd.face_centers
dist, _, _ = pp.distances.points_polygon(points, frac_pts)
faces_on_frac = np.isclose(dist, 0)

# extract the x and y components of the tractions on the fracture and make it as a vector
t_on_frac = t[np.tile(faces_on_frac, sd.dim)].reshape((sd.dim, -1))

We compute now the normal and tangential projection matrices, so that given a vector it aligns it accordingly.

In [7]:
# compute the unit normal vector to the fracture
normal = sd.face_normals[:, faces_on_frac]
normal /= np.linalg.norm(normal, axis=0)

# compute the normal and tangential projection matrix to the fracture
normal_proj = np.einsum("ik,jk->ijk", normal, normal)
tangential_proj = np.eye(3)[:, :, np.newaxis] - normal_proj

Let us compute now the normal and tangential parts of the traction and evaluate the CFF.

In [8]:
# compute the normal component of the traction
t_normal_vec = np.einsum("ijk,jk->ik", normal_proj, t_on_frac)
t_normal = np.einsum("ij,ij->j", normal, t_normal_vec)

# compute the tangential component of the traction
t_tangential_vec = np.einsum("ijk,jk->ik", tangential_proj, t_on_frac)
t_tangential = np.linalg.norm(t_tangential_vec, axis=0)

We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [9]:
# compute the Coulomb friction factor
cff = t_tangential + mu_fric * t_normal

# determine which face slips
faces_slip = cff >= 0

print(cff)
print("Number of slipping faces:", np.sum(faces_slip), "over", cff.size)

[-1.57846732e-05 -1.51719564e-05 -1.60278649e-05 -1.56746883e-05
 -1.08774372e-05 -1.06802239e-05 -1.06920859e-05 -1.04075258e-05
 -2.00543140e-05 -1.36297176e-05 -1.86126149e-05 -2.08314871e-05
 -1.25887449e-05 -1.48704018e-05 -1.81884385e-05 -1.43292557e-05
 -1.42459243e-05 -1.54701777e-05 -1.21742626e-05 -1.62504731e-05
 -1.56288441e-05 -1.56945654e-05 -1.60048425e-05 -1.66840162e-05
 -1.61348979e-05 -1.95405193e-05 -2.05336997e-05 -1.34636466e-05
 -1.51232941e-05 -1.75082143e-05 -1.49966800e-05 -1.16125854e-05
 -1.46926247e-05 -1.44986333e-05 -1.26071595e-05 -1.08125008e-05
 -1.73399242e-05 -1.24415221e-05 -2.05922036e-05 -1.76247048e-05
 -1.70892358e-05 -2.02315846e-05 -1.67731780e-05 -1.69565118e-05
 -1.88200216e-05 -1.97319327e-05 -1.29508506e-05 -1.54332209e-05
 -1.65463717e-05 -1.40389891e-05 -1.15255807e-05 -1.43873860e-05
 -1.50351322e-05 -1.16909884e-05 -1.11303713e-05 -1.30493694e-05
 -1.01530958e-05 -1.61624200e-05 -1.47125737e-05 -1.58449989e-05
 -1.62119571e-05 -1.28500

Finally, we compute the CFF and report it

In [10]:
# reshape the displacement for the export
u = np.reshape(u, (sd.dim, -1), order="F")

save = pp.Exporter(sd, "sol", folder_name="ex2")
save.write_vtu([("cell_u", u)])

In [11]:
# Consistency check
assert np.isclose(cff.sum(), -0.0025985802025895346)